In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image


class kidneyEfficientNet(nn.Module):
    def __init__(self, num_classes=44):
        super(kidneyEfficientNet, self).__init__()

        self.base_model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)
        self.base_model.classifier = nn.Identity()

        self.classifier = nn.Sequential(
            nn.BatchNorm1d(1536),
            nn.Linear(1536, 256, bias=True),
            nn.ReLU(),
            nn.Dropout(0.45),
            nn.Linear(256, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = kidneyEfficientNet(num_classes=44).to(device)

model.load_state_dict(torch.load("/content/brain_tumor_model.pth_epoch8.pth", map_location=device))
model.eval()


def predict_image(image_path, model, class_names, device):
    transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)

    return class_names[predicted_class.item()]


In [4]:
class_names = ['Cyst', 'Normal', 'Stone', 'Tumor']
model.eval()
image_path = "/content/Cyst- (10).jpg"
predicted_label = predict_image(image_path, model, class_names, device)
print(f"Predicted Class: {predicted_label}")

Predicted Class: Cyst
